##### Trial and error for dashboard 
__Larissa Voshol__  
__11/12/2023__  
__BHVF2__  

In [12]:
import numpy as np
import pandas as pd
import panel as pn
import bokeh as bk
from bokeh.plotting import figure, show
import jupyter_bokeh
pn.extension()


In [13]:
#import data
filepath_ct = '../OPC_CT_radiomics_TCIA.csv'
filepath_data = '../OPC_data_clinical.csv'

#make dataframes good
df_ct = pd.read_csv(filepath_ct)
del df_ct[df_ct.columns[0]]
df_ct.set_index("patient", inplace=True)
df_data = pd.read_csv(filepath_data)
df_data.set_index("patient", inplace=True)


In [14]:
basic_information = 0 
shape_features = 23
first_order_features = 38
GLCM_features = 55 
GLDM_features = 79
GLRLM_features = 93 
GLZSM_features = 109 
NGTDM_features = 125

start website

In [78]:
from bokeh.transform import factor_cmap
df_shape = df_ct.iloc[:,shape_features:first_order_features-1]

def render_plot(x, y, z, title):
    #grap group column
    group = df_data.loc[:,z]
    #paste to the frame with the data for the plot
    df = pd.concat([df_shape, group], axis=1)
    #unique factors
    uniq = df[z].unique()

    if len(uniq) <= 2:
        color = factor_cmap(z, ['#1f77b4', '#aec7e8'], uniq)
    if len(uniq) <=20:
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    else:
        z=""
        color=""
        

    p = figure(title = title)
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    p.scatter(x, y, source=df, color=color, legend_field=z)
    return p

In [75]:
def click_graph_btn(event):
    if not event:
        return
    body.append( pn.layout.Divider()) 
    body.append(pn.Row(
        pn.Column('# Make a graph', make_graph_input, graph_btn, pn.bind(click_graph_btn, graph_btn)),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title), title='Graph')
    ))
    

In [73]:
listy= [["nog", "meer", ["kan", "gekker"]],"meui","dikke", "priem"]

del listy[0][1]

print(type(body))

<class 'panel.layout.base.Column'>


In [79]:
#options for group
df_nonan = df_data.dropna(axis=1)

#select variables for graph
select_feature1 = pn.widgets.Select(name='Variable for x-axis', options=list(df_shape))
select_feature2 = pn.widgets.Select(name='Variable for y-axis', options=list(df_shape))
select_group = pn.widgets.Select(name='Based on wich group', options=list(df_nonan))
input_title = pn.widgets.TextInput(name='Graph title', placeholder='Enter a string here...')
graph_btn = pn.widgets.Button(name='Make new Graph', button_type='primary')

#click on buttons
graph_btn.on_click(click_graph_btn)

#Instantiate the template with widgets displayed in the sidebar
dashboard = pn.template.BootstrapTemplate(
    title='PlotRadionomics',
    sidebar=[]
)

#layout for main
make_graph_input =  pn.Card(select_feature1, select_feature2, select_group, input_title, title="Select your features")
body = pn.Column(pn.Row(
        pn.Column('# Make a graph', make_graph_input, graph_btn),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title), title='Graph'))
    )

# Append a layout to the main area, to demonstrate the list-like API
dashboard.main.append(body)


dashboard.show();


Launching server at http://localhost:57402


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : LegendItem(id='fefa7886-712b-4a58-b7f3-2bc4f73266a7', ...)
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : fill_color='' [no close matches], hatch_color='' [no close matches], line_color='' [no close matches] {renderer: GlyphRenderer(id='811ee355-84a6-45c3-a56f-5a629593557e', ...)}
ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'button_click', 'values': {'type': 'map', 'entries': [['model', {'id': 'd5c07101-32c7-4aee-9a43-67f73d93bda2'}]]}}}]} 
 error: WebSocketClosedError()
Traceback (most rece

2023-12-13 16:02:12,501 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,501 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,501 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,501 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,501 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,506 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,506 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,506 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,506 WARNING: panel.io.document - Failed sending message as connection was closed
2023-12-13 16:02:12,506 WARNING: panel.io.document - Failed sending message as connection w

Task exception was never retrieved
future: <Task finished name='Task-49711' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at c:\Users\Laris\Documents\repos\p_2.2\.venv\Lib\site-packages\tornado\websocket.py:1085> exception=WebSocketClosedError()>
Traceback (most recent call last):
  File "c:\Users\Laris\Documents\repos\p_2.2\.venv\Lib\site-packages\tornado\websocket.py", line 1087, in wrapper
    await fut
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Laris\Documents\repos\p_2.2\.venv\Lib\site-packages\tornado\websocket.py", line 1089, in wrapper
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
c:\Users\Laris\Documents\repos\p_2.2\.venv\Lib\site-packages\bokeh\document\document.py:420: RuntimeWarning: coroutine 'WSHandler.send_message' was never awaited
  gc.collect()
